In [1]:
import numpy as np
gene = ["U", "U", "I", "I", "U", "C", "C", "F", "F", "C", "C"]

In [2]:
class Node:
  def __init__(self,x,index,climit,p=None,lc=None,rc=None):

    self.data=x
    self.index=index
    self.parent=p
    self.lchild=lc
    self.rchild=rc
    self.max_child=climit
    self.nchild=0

In [3]:
def queue_tree(gene):
  split_index = int((len(gene)-1)/2)
  head = gene[:split_index]
  tail = gene[split_index:]
  queue = []
  root = Node(gene[0], 0, 2)
  queue.append(root)
  index = 0
  for i in range(1,len(head)):
    while (queue[index].nchild >= queue[index].max_child):
      index += 1
    child = Node(head[i], i, 2, queue[index])
    if queue[index].nchild == 0:
      queue[index].lchild = child
    elif queue[index].nchild == 1:
      queue[index].rchild = child
    else:
      print("Error")

    queue[index].nchild += 1
    queue.append(child)

  number_of_leaves = int((len(head)+1)/2)
  depth = np.log2(len(head))
  first_leaf = int(2**depth - 1)
  post_order_traversal = []
  for i in range(first_leaf, len(head)):
    post_order_traversal.append(i)
  for i in range(number_of_leaves - 1, first_leaf):
    post_order_traversal.append(i)

  leaf_index = 0
  for i in range(len(tail)):
    while (queue[post_order_traversal[leaf_index]].nchild >= queue[post_order_traversal[leaf_index]].max_child):
      leaf_index += 1
    child = Node(tail[i], i+len(head), 2, queue[post_order_traversal[leaf_index]])
    if queue[post_order_traversal[leaf_index]].nchild == 0:
      queue[post_order_traversal[leaf_index]].lchild = child
    elif queue[post_order_traversal[leaf_index]].nchild == 1:
      queue[post_order_traversal[leaf_index]].rchild = child
    else:
      print("Error")
    queue[post_order_traversal[leaf_index]].nchild += 1
    queue.append(child)

  return queue

In [4]:
queue = queue_tree(gene)
for i in range(len(gene)):
  print(f"{i}: {queue[i].data}")

0: U
1: U
2: I
3: I
4: U
5: C
6: C
7: F
8: F
9: C
10: C


In [5]:
def intersection(list1, list2):
  list3 = [value for value in list1 if value in list2]
  return list3

def decode(queue, index):
  root = queue[index]
  if root.nchild == 0:
    return root.data
  lchild = root.lchild
  rchild = root.rchild
  cnn_decoded = []
  match root.data:
    case "U":
      #print("U")
      lchild_set = decode(queue, lchild.index)
      rchild_set = decode(queue, rchild.index)
      #print(f"{lchild.index}: {lchild_set}")
      #print(f"{rchild.index}: {rchild_set}")
      cnn_decoded.extend(lchild_set)
      cnn_decoded.extend(rchild_set)
    case "I":
      #print("I")
      lchild_set = decode(queue, lchild.index)
      rchild_set = decode(queue, rchild.index)
      #print(f"{lchild.index}: {lchild_set}")
      #print(f"{rchild.index}: {rchild_set}")
      cnn_decoded = intersection(lchild_set, rchild_set)
  return cnn_decoded

In [ ]:
sorted(decode(queue, 0))

['C', 'C', 'F', 'F']

In [15]:
import keras
from keras.layers import Conv2D

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

from keras import Sequential


t=0

def buildconv(fs,ks,ds,c_act,d_act):
    #fs- Filter size
    #ks-Kernel size
    #ds-Dense layer size
    #c_act- Convolutional Layer Activation
    #d_act- Dense Layer Activation
    model = Sequential()
    c_f=0
    for i in sorted(decode(queue, 0)):


        #print(t)
        if i=='C':

            model.add(Conv2D(filters = fs, kernel_size = ks,padding = 'Same',activation ='relu', input_shape = (28,28,1)))

        if i=='F':
            if c_f==0:

                model.add(Flatten())

                c_f+=1
            model.add(Dense(ds, activation = "relu"))

    keras.utils.plot_model(model,to_file='model_architecture.png')



In [17]:
buildconv(32,(5,5),256,'relu','relu')